In [ ]:
%pip install pandas
%pip install lxml

In [5]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
SCORE_DIR = "data/scores"

In [7]:
box_scores = os.listdir(SCORE_DIR)

In [8]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [9]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [12]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id":"line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [13]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {"id":f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df    

In [14]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [15]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)
    summary = pd.concat(summaries, axis = 1).T

    game = pd.concat([summary, line_score], axis = 1)

    game["home"] = [0, 1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis = 1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    l
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 10205
200 / 10205
300 / 10205
400 / 10205
500 / 10205
600 / 10205
700 / 10205
800 / 10205
900 / 10205
1000 / 10205
1100 / 10205
1200 / 10205
1300 / 10205
1400 / 10205
1500 / 10205
1600 / 10205
1700 / 10205
1800 / 10205
1900 / 10205
2000 / 10205
2100 / 10205
2200 / 10205
2300 / 10205
2400 / 10205
2500 / 10205
2600 / 10205
2700 / 10205
2800 / 10205
2900 / 10205
3000 / 10205
3100 / 10205
3200 / 10205
3300 / 10205
3400 / 10205
3500 / 10205
3600 / 10205
3700 / 10205
3800 / 10205
3900 / 10205
4000 / 10205
4100 / 10205
4200 / 10205
4300 / 10205
4400 / 10205
4500 / 10205
4600 / 10205
4700 / 10205
4800 / 10205
4900 / 10205
5000 / 10205
5100 / 10205
5200 / 10205
5300 / 10205
5400 / 10205
5500 / 10205
5600 / 10205
5700 / 10205
5800 / 10205
5900 / 10205
6000 / 10205
6100 / 10205
6200 / 10205
6300 / 10205
6400 / 10205
6500 / 10205
6600 / 10205
6700 / 10205
6800 / 10205
6900 / 10205
7000 / 10205
7100 / 10205
7200 / 10205
7300 / 10205
7400 / 10205
7500 / 10205
7600 / 10205
7700 / 10205
7800 / 1

In [26]:
games_df = pd.concat(games, ignore_index=True)

In [49]:
pd.set_option('display.max_columns', None)

games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,+/-,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,mp_max,mp_max,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,index_opp,mp_opp,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,+/-_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,mp_max_opp,mp_max_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,NaN,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,100.0,111.2,98.6,NaN,NaN,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.000,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,NaN,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,100.0,98.6,111.2,NaN,NaN,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.0,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,10.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,NaN,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,100.0,98.6,111.2,NaN,NaN,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.000,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,10.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,94,1,0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,NaN,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,100.0,111.2,98.6,NaN,NaN,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,NaN,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,NaN,NaN,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.000,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,1,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,NaN,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,NaN,NaN,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,NaN,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,NaN,NaN,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.000,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,0,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,NaN,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,NaN,NaN,12.0,22.0,0.571

In [53]:
games_df.to_csv("nba_games.csv")